In [1]:
import pandas as pd

In [2]:
data=pd.read_pickle('2317_vector_full_data.pkl')

In [4]:
#使用方式 與格式 1
print(data['Info'])
print(data['Words'][1000:1002])
print(data['Vectors'][1000:1002])
print(data['Label'][1000:1002])

0                               董事會決議日期   
1                     發放股利種類及金額 現金股利美金   元
2                                其他應敘明事項 無
3                               董事會決議日期   
4         發放股利種類及金額 \n股票股利美金   元 配發新股  股  
                       ...                
49523                         本次交易為關係人交易 否
49524                        董事會通過日期 \n不適用
49525                        監察人承認日期 \n不適用
49526                    本次交易會計師出具非合理性意見 否
49527                           其他敘明事項 \n無
Name: Info, Length: 49528, dtype: object
1000                 [董事會, 通過, 日期, 不, 適用]
1001    [監察人, 承認, 審計, 委員會, 同意, 日期, 不, 適用]
Name: Words, dtype: object
1000    [[0.060508467, 0.3024362, 1.744864, -1.9498981...
1001    [[3.9632685, -2.7145953, 2.6416998, -1.0490379...
Name: Vectors, dtype: object
1000    0.0
1001    0.0
Name: Label, dtype: object


In [31]:
#使用方式 與格式 2
print(data.Info)
print(data.Words)
print(data.Vectors)
print(data.Label[1000:1002])

0                               董事會決議日期   
1                     發放股利種類及金額 現金股利美金   元
2                                其他應敘明事項 無
3                               董事會決議日期   
4         發放股利種類及金額 \n股票股利美金   元 配發新股  股  
                       ...                
49523                         本次交易為關係人交易 否
49524                        董事會通過日期 \n不適用
49525                        監察人承認日期 \n不適用
49526                    本次交易會計師出具非合理性意見 否
49527                           其他敘明事項 \n無
Name: Info, Length: 49528, dtype: object
0                                  [董事會, 決議, 日期]
1                [發放, 股利, 種類, 金額, 現金, 股利, 美金, 元]
2                                 [應, 敘明, 事項, 無]
3                                  [董事會, 決議, 日期]
4        [發放, 股利, 種類, 金額, 股票, 股利, 美金, 配發, 新股, 股]
                          ...                   
49523                     [次, 交易, 為, 關係人, 交易, 否]
49524                       [董事會, 通過, 日期, 不, 適用]
49525                       [監察人, 承認, 日期, 不, 適用]
49526            [次, 交易, 會計師, 出具, 非, 合理性, 意見,

In [3]:
copydata=data.copy()
copydata['newLabel']=0
copydata['Label']=copydata['Label'].astype(float)
for i in range(len(data)):
    if(copydata['Label'][i]>=0.3 and copydata['Label'][i]<0.7):
        copydata['newLabel'][i]=3
    elif(copydata['Label'][i]>=0.7 and copydata['Label'][i]<=1.0):
        copydata['newLabel'][i]=4
    elif(copydata['Label'][i]==0.0):
        copydata['newLabel'][i]=2
    elif(copydata['Label'][i]>-0.7 and copydata['Label'][i]<=-0.2):
        copydata['newLabel'][i]=1
    elif(copydata['Label'][i]<=-0.7 and copydata['Label'][i]>=-1.0):
        copydata['newLabel'][i]=0
copydata['newLabel']        


C:\Users\user\AppData\Local\Temp\ipykernel_5812\1167576641.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  copydata['newLabel'][i]=1
C:\Users\user\AppData\Local\Temp\ipykernel_5812\1167576641.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  copydata['newLabel'][i]=2
C:\Users\user\AppData\Local\Temp\ipykernel_5812\1167576641.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  copydata['newLabel'][i]=3
C:\Users\user\AppData\Loca

0        1
1        1
2        1
3        1
4        1
        ..
49523    3
49524    3
49525    3
49526    3
49527    3
Name: newLabel, Length: 49528, dtype: int64

In [20]:
type(copydata['Label'][0])

numpy.float64

In [21]:
copydata['Label']=copydata['Label'].astype(int)

In [4]:
copydata['newLabel'][10000]

1

In [5]:
data['Label'][10000]

'-0.4799997965494791'

In [83]:
len(copydata['Vectors'][0][2])

80

In [59]:
copydata['Info'][1]

' 發放股利種類及金額 現金股利美金   元'

In [12]:
from tqdm import tqdm
counter=[]
temp=[]
for i in tqdm(range(len(copydata))):
    temp=copydata['Vectors'][i][0].tolist()
    for j in range(len(copydata['Vectors'][i])):
        temp=temp+copydata['Vectors'][i][j].tolist()
    counter.append(temp)
# print(np.array(copydata['Vectors'][5000]).flatten().size)

100%|██████████████████████████████████████████████████████████████████████████| 49528/49528 [00:07<00:00, 6526.85it/s]


In [98]:
sentiment=[]
for i in range(len(copydata)):
    sentiment.append(copydata['newLabel'][i])

In [70]:
import numpy as np
np.array(copydata['Vectors'][0]).flatten().size 

240

In [91]:
from sklearn.model_selection import train_test_split # to split test and train
from tensorflow.keras.utils import to_categorical # to categorical data
y = to_categorical(sentiment).astype(int)
y

array([[0, 1, 0, 0, 0],
       [0, 1, 0, 0, 0],
       [0, 1, 0, 0, 0],
       ...,
       [0, 0, 0, 1, 0],
       [0, 0, 0, 1, 0],
       [0, 0, 0, 1, 0]])

In [26]:
from tensorflow.keras.datasets import  imdb 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import GRU


In [115]:
np.array(sentiment).shape

(49528,)

In [104]:
x=sequence.pad_sequences(np.array(counter),maxlen=512,dtype='float32')
# for i in range(len(copydata)):
#     prinlen(len(counter[0][0]))
X_train.shape,y_train.shape ,x.shape

C:\Users\user\AppData\Local\Temp\ipykernel_5812\4105218851.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  x=sequence.pad_sequences(np.array(counter),maxlen=512,dtype='float32')


((47051, 720), (47051, 5), (49528, 512))

In [105]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.05)

In [108]:
x,y

(array([[ 0.        ,  0.        ,  0.        , ...,  0.06664681,
          2.2931826 , -0.6105696 ],
        [-0.2988216 ,  0.14024633,  3.5166476 , ..., -1.2650552 ,
         -3.0302546 , -1.9531665 ],
        [ 0.        ,  0.        ,  0.        , ...,  0.52202034,
         -1.4835407 ,  0.8500348 ],
        ...,
        [ 0.        ,  0.        ,  0.        , ...,  0.48199058,
         -1.3232802 ,  1.6739393 ],
        [-0.404489  ,  1.4311732 ,  2.3006833 , ..., -0.763865  ,
          6.255383  , -1.4383389 ],
        [ 0.        ,  0.        ,  0.        , ...,  0.52202034,
         -1.4835407 ,  0.8500348 ]], dtype=float32),
 array([[0, 1, 0, 0, 0],
        [0, 1, 0, 0, 0],
        [0, 1, 0, 0, 0],
        ...,
        [0, 0, 0, 1, 0],
        [0, 0, 0, 1, 0],
        [0, 0, 0, 1, 0]]))

In [133]:
from tensorflow.keras.initializers import Constant
from tensorflow.keras import layers
model = Sequential()
model.add(Embedding(49528,151,input_length=512))
# model.add(layers.SimpleRNN(80))
# model.add(layers.Flatten())
model.add(GRU(151))
model.add(Dense(5,activation='softmax'))
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential_35"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_34 (Embedding)     (None, 512, 151)          7478728   
_________________________________________________________________
gru_14 (GRU)                 (None, 151)               137712    
_________________________________________________________________
dense_22 (Dense)             (None, 5)                 760       
Total params: 7,617,200
Trainable params: 7,617,200
Non-trainable params: 0
_________________________________________________________________
None


In [135]:
from tensorflow.keras.callbacks import TensorBoard
import os
# log_dir = os.path.join('Logs')
# tb_callback = TensorBoard(log_dir=log_dir)
model.fit(x,y, epochs=16)

Epoch 1/16


InvalidArgumentError: 2 root error(s) found.
  (0) Invalid argument:  indices[22,257] = -4 is not in [0, 49528)
	 [[node sequential_35/embedding_34/embedding_lookup (defined at C:\Users\user\AppData\Local\Temp\ipykernel_5812\2939073853.py:5) ]]
	 [[Adam/Adam/update/AssignSubVariableOp/_35]]
  (1) Invalid argument:  indices[22,257] = -4 is not in [0, 49528)
	 [[node sequential_35/embedding_34/embedding_lookup (defined at C:\Users\user\AppData\Local\Temp\ipykernel_5812\2939073853.py:5) ]]
0 successful operations.
0 derived errors ignored. [Op:__inference_train_function_58769]

Errors may have originated from an input operation.
Input Source operations connected to node sequential_35/embedding_34/embedding_lookup:
 sequential_35/embedding_34/embedding_lookup/57533 (defined at D:\anaconda\envs\mediapipe\lib\contextlib.py:113)

Input Source operations connected to node sequential_35/embedding_34/embedding_lookup:
 sequential_35/embedding_34/embedding_lookup/57533 (defined at D:\anaconda\env

In [132]:
for i in range(len(X_train)):
    if(len(X_train[i])!=512):
        print('fail')

In [46]:
y_train

array([[0, 0, 1, 0, 0],
       [0, 0, 1, 0, 0],
       [0, 0, 1, 0, 0],
       ...,
       [0, 1, 0, 0, 0],
       [0, 0, 1, 0, 0],
       [0, 0, 0, 1, 0]])

In [54]:
np.array(sentiment).shape


(49528,)

In [141]:
test.iloc[3895]

Unnamed: 0                                                 3895
Info                                             本次交易為關係人交易 \n否
Label                                                       0.0
Words                             ['次', '交易', '為', '關係人', '交易']
Vectors       [array([-0.6673892 ,  3.3785224 , -3.5420296 ,...
newLabel                                                      2
Name: 3895, dtype: object

In [126]:
copydata['Info'][100]

' 交付或付款條件 含付款期間及金額  契約限制條款及其他重要\n約定事項 \n無'